In [1]:
%load_ext autoreload
%autoreload 2

In [111]:
from models import CTCNet
from utils import make_grid
import torch
import time

In [74]:
# model_param_grid
model_params = {"input_size": [28 * 28],
                    "output_size": [10],
                    "ctx_layer_size": [128],
                    "thal_layer_size": [64],
                    "thalamocortical_type": [None, "additive", "multi_pre_activation", "multi_post_activation"],
                    "thal_reciprocal": [True, False],
                    "thal_to_readout": [True, False],
                    "thal_per_layer": [True, False]}

In [80]:
model_param_grid = make_grid(model_params)

In [70]:
combination_idx = 0
for param_name, param_val in model_param_grid.items():

    model = CTCNet(input_size=input_size,
                output_size=output_size,
                ctx_layer_size=ctx_layer_size,
                thal_layer_size=thal_layer_size,
                thalamocortical_type="multiplicative_post_activation",
                thal_reciprocal=True, 
                thal_to_readout=True, 
                thal_per_layer=False)

{'input_size': 0,
 'output_size': 0,
 'ctx_layer_size': 0,
 'thal_layer_size': 0,
 'thalamocortical_type': 0,
 'thal_reciprocal': 0,
 'thal_to_readout': 0,
 'thal_per_layer': 0}

In [21]:
input_size = 28 * 28
output_size = 10
ctx_layer_size = 10
thal_layer_size = 10
thalamocortical_type=None # None, multiplicative, or additive
thal_reciprocal=True # True or False
thal_to_readout=True # True or False
thal_per_layer=False # if no, mixing from cortical layers
fan_in_thal=None
fan_in_ctx=None

In [50]:
model = CTCNet(input_size=input_size,
                output_size=output_size,
                ctx_layer_size=ctx_layer_size,
                thal_layer_size=thal_layer_size,
                thalamocortical_type="multiplicative_post_activation",
                thal_reciprocal=True, 
                thal_to_readout=True, 
                thal_per_layer=False)

In [51]:
model.summary()

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Linear: 2-1                       210
|    └─ReLU: 2-2                         --
├─Linear: 1-2                            110
├─Linear: 1-3                            110
├─Linear: 1-4                            110
├─Sequential: 1-5                        --
|    └─Linear: 2-3                       110
|    └─ReLU: 2-4                         --
├─Sequential: 1-6                        --
|    └─Linear: 2-5                       110
|    └─ReLU: 2-6                         --
├─Sequential: 1-7                        --
|    └─Linear: 2-7                       110
|    └─Softmax: 2-8                      --
Total params: 870
Trainable params: 870
Non-trainable params: 0


In [83]:
from torchvision import datasets, transforms

Setting backend.
Using cpu device.


In [84]:
datasets.FashionMNIST

torchvision.datasets.mnist.FashionMNIST

---

In [104]:
def create_data_loaders(dataset, norm,, batch_size, save_path):

    # choose normalisation method
    if norm == "normalise":
        transform = transforms.Compose([
            transforms.ToTensor(), # scale values to lie in range [0, 1]
        ])
    elif norm == "standardise":
                transform = transforms.Compose([
            transforms.ToTensor(), 
            transforms.Normalize((0.), (1.,))  # standarise values to have zero mean and unit SD
        ])
                
    metadata = {}
    # load dataset and apply normalisation
    if dataset == "MNIST":
        trainset = datasets.MNIST(root=save_path,
                                  train=True,
                                  download=True,
                                  transform=transform)
        testset = datasets.MNIST(root=save_path,
                                  train=False,
                                  download=True,
                                  transform=transform)
        metadata["classes"] = trainset.classes
    elif dataset == "FashionMNIST":
        trainset = datasets.FashionMNIST(root=save_path,
                                         train=True,
                                         download=True,
                                         transform=transform)
        testset = datasets.FashionMNIST(root=save_path,
                                        train=False,
                                        download=True,
                                        transform=transform)
        
        metadata["classes"] = trainset.classes
    # create loaders
    trainset_loader = torch.utils.data.DataLoader(dataset = trainset,
                                batch_size = batch_size,
                                shuffle = True)
    testset_loader = torch.utils.data.DataLoader(dataset = testset,
                                batch_size = batch_size,
                                shuffle = True)

    return trainset_loader, testset_loader, metadata

In [108]:
norm = "normalise"
dataset = "FashionMNIST"
save_path = "/Users/patmccarthy/Documents/BurstCCN/Data2"
batch_size = 32

In [109]:
trainset_loader, testset_loader, metadata = create_data_loaders(dataset=dataset,
                                                      norm=norm,
                                                      save_path=save_path,
                                                      batch_size=batch_size)

100%|██████████| 26421880/26421880 [00:01<00:00, 19463890.38it/s]


Extracting /Users/patmccarthy/Documents/BurstCCN/Data2/FashionMNIST/raw/train-images-idx3-ubyte.gz to /Users/patmccarthy/Documents/BurstCCN/Data2/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 1599396.42it/s]

Extracting /Users/patmccarthy/Documents/BurstCCN/Data2/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /Users/patmccarthy/Documents/BurstCCN/Data2/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 11821870.57it/s]


Extracting /Users/patmccarthy/Documents/BurstCCN/Data2/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /Users/patmccarthy/Documents/BurstCCN/Data2/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5105764.24it/s]

Extracting /Users/patmccarthy/Documents/BurstCCN/Data2/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/patmccarthy/Documents/BurstCCN/Data2/FashionMNIST/raw



In [110]:
metadata

{'classes': ['T-shirt/top',
  'Trouser',
  'Pullover',
  'Dress',
  'Coat',
  'Sandal',
  'Shirt',
  'Sneaker',
  'Bag',
  'Ankle boot']}

In [101]:
            
mnist_trainset = datasets.MNIST(root="/Users/patmccarthy/Documents/BurstCCN/Data2", train=True, download=True)
print("Dataset downloaded.")

Dataset downloaded.


In [103]:
mnist_trainset.classes

['0 - zero',
 '1 - one',
 '2 - two',
 '3 - three',
 '4 - four',
 '5 - five',
 '6 - six',
 '7 - seven',
 '8 - eight',
 '9 - nine']

In [ ]:
# Download MNIST
print("Downloading dataset.")
if norm == "normalise":
    transform = transforms.Compose([
        transforms.ToTensor(), # scale values to lie in range [0, 1]
    ])
elif norm == "standardise":
            transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize((0.), (1.,))  # normalise values to 
    ])
            
mnist_trainset = datasets.MNIST(root="/Users/patmccarthy/Documents/BurstCCN/Data2", train=True, download=True, transform=transform)
print("Dataset downloaded.")

# Load MNIST
data_loader = torch.utils.data.DataLoader(dataset = mnist_trainset,
                                    batch_size = HP["BATCH_SIZE"],
                                    shuffle = True)

# Initialise model
print("Initialising model.")
model = AE(encoding_dim=HP["ENCODING_DIM"])
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                            lr = HP["LEARNING_RATE"])
print("Done initialising.")

start_time = time.time()

# Train model
print("Training...")
losses_epochs = []
for epoch in range(HP["EPOCHS"]):
    print(f"Beginning epoch {epoch+1}/{HP['EPOCHS']}")
    losses = train(data_loader=data_loader, model=model, loss_fn=loss_function, optimizer=optimizer)
    losses_epochs.append(losses)
    print(f"Epoch {epoch+1}/{HP['EPOCHS']} done")
end_time = time.time() - start_time
print(f"Finished training in {end_time:.2f} seconds.")

In [ ]:
def train(model, optimizer, loss, num_epochs, verbose=True):
    """Train model and regularly evaluate on validation set."""
    
    start_time = time.time()
    print("Training...")
    losses_epochs = []
    for epoch in range(num_epochs):
        print(f"Beginning epoch {epoch+1}/{num_epochs}")
        losses = train(data_loader=data_loader, model=model, loss_fn=loss_function, optimizer=optimizer)
        losses_epochs.append(losses)
        print(f"Epoch {epoch+1}/{num_epochs} done")
    end_time = time.time() - start_time
    print(f"Finished training in {end_time:.2f} seconds.")



def train_one_epoch(model, trainset_loader, valset_loader, optimizer, loss_fn, num_epochs, device, loss_track_step):
    """Train model for one epoch."""

    size = len(data_loader.dataset)
    model.train()
    losses = []
    for batch, (X, _) in enumerate(data_loader):
        # move data to device where model will be trained
        X = X.to(device)

        # compute error
        _, X_recon = model(X)
        loss = loss_fn(X_recon, X)

        # backprop
        loss.backward()  # compute gradients
        optimizer.step()  # update params
        optimizer.zero_grad()  # ensure not tracking gradients fo next iteration

        # print loss every Nth batch
        if batch % loss_track_step == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(
                f"training batch {batch+1}, loss: {loss:.3f}, {current}/{size} datapoints"
            )
            losses.append(loss)

    return losses

In [ ]:
# define model, optimiser, loss
model = CTCNet(input_size=input_size,
                output_size=output_size,
                ctx_layer_size=ctx_layer_size,
                thal_layer_size=thal_layer_size,
                thalamocortical_type="multiplicative_post_activation",
                thal_reciprocal=True, 
                thal_to_readout=True, 
                thal_per_layer=False)
loss_function = torch.nn.CrossEntropyLoss()

model = AE(encoding_dim=HP["ENCODING_DIM"])
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                            lr = HP["LEARNING_RATE"])

---
Routine to train single model

In [ ]:
hyperparams = {...}